In [1]:
versioninfo()

Julia Version 1.9.0-DEV.862
Commit b11ccae719* (2022-06-29 03:11 UTC)
Platform Info:
  OS: Linux (x86_64-redhat-linux)
  CPU: 24 × Intel(R) Xeon(R) Gold 6226 CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.5 (ORCJIT, cascadelake)
  Threads: 1 on 24 virtual cores
Environment:
  LD_LIBRARY_PATH = /opt/nec/ve/nlc/2.3.0/lib


In [7]:
home=ENV["HOME"]
run(`$home/julia/julia/usr/tools/llc --version`)

LLVM (http://llvm.org/):
  LLVM version 14.0.5jl
  Optimized build.
  Default target: x86_64-redhat-linux
  Host CPU: cascadelake

  Registered Targets:
    amdgcn  - AMD GCN GPUs
    bpf     - BPF (host endian)
    bpfeb   - BPF (big endian)
    bpfel   - BPF (little endian)
    nvptx   - NVIDIA PTX 32-bit
    nvptx64 - NVIDIA PTX 64-bit
    r600    - AMD GPUs HD2XXX-HD6XXX
    ve      - VE
    wasm32  - WebAssembly 32-bit
    wasm64  - WebAssembly 64-bit
    x86     - 32-bit X86: Pentium-Pro and above
    x86-64  - 64-bit X86: EM64T and AMD64


Process(`/home/manabu/julia/julia/usr/tools/llc --version`, ProcessExited(0))

In [8]:
using LLVM

In [11]:
using Pkg
Pkg.status("LLVM")

Status `~/.julia/environments/v1.9/Project.toml`
  [929cbde3] LLVM v4.14.0


In [12]:
using GPUCompiler
Pkg.status("GPUCompiler")

Status `~/.julia/environments/v1.9/Project.toml`
  [61eb1bfa] GPUCompiler v0.16.1


In [23]:
function kernel()
    ccall("extern sleep", llvmcall, Cvoid, (Int64,), UInt64(5))
    return nothing
end

kernel (generic function with 1 method)

In [37]:
source = FunctionSpec(kernel, Tuple{}, true, "veo_sleep")

kernel veo_sleep()

In [38]:
struct VECompilerTarget <: AbstractCompilerTarget end
struct VECompilerParams <: AbstractCompilerParams end

In [39]:
job = CompilerJob(VECompilerTarget(), source, VECompilerParams())

CompilerJob of kernel veo_sleep() for VECompilerTarget

In [40]:
method_instance, mi_meta = GPUCompiler.emit_julia(job)

(MethodInstance for kernel(), ())

In [41]:
GPUCompiler.llvm_triple(::VECompilerTarget) = "ve-unknown-linux-gnu"
GPUCompiler.runtime_slug(::GPUCompiler.CompilerJob{VECompilerTarget}) = "ve_kernel"

In [42]:
ir, ir_meta = GPUCompiler.emit_llvm(job, method_instance; ctx=JuliaContext())

(; ModuleID = 'start'
source_filename = "start"
target datalayout = "e-m:e-i64:64-n32:64-S128-v64:64:64-v128:64:64-v256:64:64-v512:64:64-v1024:64:64-v2048:64:64-v4096:64:64-v8192:64:64-v16384:64:64"
target triple = "ve-unknown-linux-gnu"

define void @_Z15julia_veo_sleep() local_unnamed_addr #0 !dbg !5 {
top:
  call void @sleep(i64 5), !dbg !8
  ret void, !dbg !9
}

declare void @sleep(i64) local_unnamed_addr

attributes #0 = { "probe-stack"="inline-asm" }

!llvm.module.flags = !{!0, !1}
!llvm.dbg.cu = !{!2}
!julia.kernel = !{!4}

!0 = !{i32 2, !"Dwarf Version", i64 4}
!1 = !{i32 2, !"Debug Info Version", i32 3}
!2 = distinct !DICompileUnit(language: DW_LANG_Julia, file: !3, producer: "julia", isOptimized: true, runtimeVersion: 0, emissionKind: LineTablesOnly, nameTableKind: None)
!3 = !DIFile(filename: "In[23]", directory: ".")
!4 = !{void ()* @_Z15julia_veo_sleep}
!5 = distinct !DISubprogram(name: "kernel", linkageName: "julia_kernel_3753", scope: null, file: !3, line: 1, type: !6, s

In [43]:
obj, obj_meta = GPUCompiler.emit_asm(job, ir; format=LLVM.API.LLVMObjectFile, validate=false)

("\x7fELF\x02\x01\x01\0\0\0\0\0\0\0\0\0\x01\0\xfb\0\x01\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\xa0\x03\0\0\0\0\0\0\0\0\0\0@\0\0\0\0\0@\0\f\0\x01\0\0\0\0\0\x8b\0\t\x11\b\0\0\0\x8b\0\n\x11\0\0\0\0\x8b\0\tE\x10\xff\xff\xff\x8b\0\v\x06H\0\0\0\x88\x8b5\x18\x18\0\0\0\x8e\0=\x01\0\0\0\0\x80\0>E;\x01\0\0\0\0?\x06\0\0\0\0\xbd\x03?1\b\0\0\0\xbd\x03\b1\x10\0\0\0\xbd\x03\v1\0\0\0\0\0\0\0?\0\0\0\0\xbe\0\0E\0\0\0\0\0\0\0\x06\0\0\0\0`\x80\0D\0\0\0\0\x80\0\x8c\x06\0\0\0\0\0\x05\0E\0\0\0\0\x8c\0\n\b\0\0\0\0\x89\0\vE\b\0\0\0\x8b\0\n\x01\0\0\0\0\x8b\0\t\x01\0\0\0\0\x8a\0?\x19\x01\x11\0%\x0e\x13\x05\x03\x0e\x10\x17\e\x0e\x11\x01\x12\x06\0\0\0&\0\0\0\x04\0\0\0\0\0\b\x01\0\0\0\0\x1f\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\xb0\0\0\0julia\0In[23]\0.\08\0\0\0\x04\0\x1e\0\0\0\b\x01\x01\xfb\x0e\r\0\x01\x01\x01\x01\0\0\0\x01\0\0\x01\0In[23]\0\0\0\0\0\0\t\x02\0\0\0\0\0\0\0\0\x01\nɃ\x02\x01\0\x01\x01\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0\x1a\0\0\0\x04\0\xf1\xff\0\0\0\0\0\0\0\0\0\0\0\0\0\0\0

In [44]:
vso = mktemp() do path_o, io_o
    write(io_o, obj)
    flush(io_o)
    run(`/opt/nec/ve/bin/nld -shared -o test2.so $path_o`)
end

Process(`/opt/nec/ve/bin/nld -shared -o test2.so /tmp/jl_4hd6nE`, ProcessExited(0))